In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests

In [ ]:
df = pd.read_csv(file, sep='\t',low_memory=False)
sdf = df[['CHROM', 'POS', 'REF', 'ALT']].copy()
sdf['CHROM'] = sdf['CHROM'].str.replace('chr', '')
sdf = sdf.astype('str')
sdf.drop_duplicates(keep="first", inplace=True)
sdf.head()

In [ ]:
#read in a given file to get cancervar predictions 
#post running the  code, you can read in the file and plot the results
out="file" # you have to change this
with open(out, 'w+') as o:
    for index, row in sdf.iterrows():
        chrom = row['CHROM']
        pos = row['POS']
        ref = row['REF']
        alt = row['ALT']
        url = "http://cancervar.wglab.org/api_new.php?queryType=position&build=hg19&chr=" + chrom + "&pos=" + pos + "&ref=" + ref + '&alt=' + alt
        time.sleep(.5)
        response = requests.get(url)
        code = response.status_code
        try:
            jr = response.json()
            tier = jr['Cancervar']
        except:
            tier = 'NA'
        o.write(chrom + "\t" + pos + '\t' + ref + '\t' + alt + '\t' + tier + '\n')

In [ ]:
#post running of the above code, you can read in the file and plot the results
cvi = pd.read_csv("upenn_independent_set.txt", header=None, sep='\t')

#first lets harmonize data from the cancervar calls
cv = cvi.rename({0: 'CHROM', 1: 'POS', 2: 'REF', 3: 'ALT', 4: 'CancerVar'}, axis=1)
cv['CHROM'] = 'chr' + cv['CHROM']
#'Tier I/Pathogenic: Variants of Strong Clinical Significance','Tier II/Likely Pathogenic: Variants of Potential Clinical Significance','Tier III: Variants of Unknown Clinical Significance' and 'Tier IV: Benign or Likely Benign Variants', together with detailed evidence.
#lets convert the cancervar to cpd cats
cv['CancerVar'] = cv['CancerVar'].str.split('#').str[1]
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_I_strong', 'Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_III_Uncertain', 'VUS')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_II_potential', 'Likely Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_IV_benign', 'Likely Benign')
#cv =cv[cv['CancerVar'] != 'nan'] #comment this in or out to leave NA in or out
cv.head()

In [ ]:
m = cpdr.merge(cv, how='left', on=['CHROM','POS', 'REF', 'ALT'])
m.drop_duplicates()
m.head(20)

In [ ]:
confusion_matrix = pd.crosstab(m['CATEGORIZATION'], m['CancerVar'], rownames=['Clinical Review'], colnames=['CancerVar']).apply(lambda r: r/r.sum(), axis=1)
confusion_matrix.head()

In [ ]:
cm2 = pd.crosstab(m['CATEGORIZATION'], m['comp_pred'], rownames=['Clinical Review'], colnames=['Azurify']).apply(lambda r: r/r.sum(), axis=1)
cm2.head()

In [ ]:
# THIS IS THE CANCER VAR SET WITH NO Benign USING THIS FOR PUBLICATION 
data = { 'Azurify' : [97.51,24.10,84.31,37.61],
        'CancerVar': [13.61, 41.03, 67.11, 10.11],
        'Category': ['Pathogenic', 'Likely Pathogenic', 'VUS', 'Likely Benign']}
az = pd.DataFrame(data)
az = az.set_index('Category')
az.head()

In [ ]:

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
color_red = '#990000'  
color_blue = '#011F5B'
index = az.index
column0 = az['Azurify']
column1 = az['CancerVar']
title0 = 'Azurify'
title1 = 'CancerVar'

fig, axes = plt.subplots(figsize=(10,5), ncols=2, sharey=True)
fig.tight_layout()

axes[0].barh(index, column0, align='center', color=color_blue, zorder=10)
axes[0].set_title(title0, fontsize=18, pad=15, color=color_blue, **hfont)
axes[1].barh(index, column1, align='center', color=color_red, zorder=10)
axes[1].set_title(title1, fontsize=18, pad=15, color=color_red, **hfont)
axes[0].invert_xaxis()
axes[0].set(yticks=az.index, yticklabels=az.index)
axes[0].yaxis.tick_left()
axes[0].tick_params(axis='y', colors='white') # tick color

axes[1].set_xticks([0, 20, 40, 60, 80, 100])

for label in (axes[0].get_xticklabels() + axes[0].get_yticklabels()):
    label.set(fontsize=16, color='black')
for label in (axes[1].get_xticklabels() + axes[1].get_yticklabels()):
    label.set(fontsize=16, color='black')
#plt.style.use('seaborn-white')
plt.gca().invert_yaxis()
plt.subplots_adjust(wspace=0, top=0.85, bottom=0.1, left=0.18, right=0.95)
plt.savefig('AZvsCV.pdf', dpi=600)

In [ ]:
#now compare using UC and UAB data

In [ ]:
uab_cv_aml = pd.read_csv('UAB_cancervar.txt', sep='\t')
UC_cv_all = pd.read_csv('UC_cancervar.txt', sep='\t')


In [ ]:
concord_canvervar = pd.read_csv('concord_uc_uab_cv.txt', sep='\t', header=None, names=['CHROM', 'POS', 'REF', 'ALT', 'CancerVar'])
concord_canvervar.head()

In [ ]:
#first lets harmonize data from the cancervar calls
cv = concord_canvervar.rename({0: 'CHROM', 1: 'POS', 2: 'REF', 3: 'ALT', 4: 'CancerVar'}, axis=1)
cv['CHROM'] = 'chr' + cv['CHROM']
#'Tier I/Pathogenic: Variants of Strong Clinical Significance','Tier II/Likely Pathogenic: Variants of Potential Clinical Significance','Tier III: Variants of Unknown Clinical Significance' and 'Tier IV: Benign or Likely Benign Variants', together with detailed evidence.
#lets convert the cancervar to cpd cats
cv['CancerVar'] = cv['CancerVar'].str.split('#').str[1]
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_I_strong', 'Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_III_Uncertain', 'Non-Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_II_potential', 'Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_IV_benign', 'Non-Pathogenic')
#cv =cv[cv['CancerVar'] != 'nan'] #comment this in or out to leave NA in or out
cv.head()

In [ ]:
ca = pd.read_csv('all_concordant.tsv', sep='\t')
ca.head()

In [ ]:
m = ca.merge(cv, how='left', on=['CHROM','POS', 'REF', 'ALT'])
m.drop_duplicates()


In [ ]:
confusion_matrix = pd.crosstab(m['EXT_CATEGORIZATION'], m['CancerVar'], rownames=['EXT_CATEGORIZATION'], colnames=['CancerVar']).apply(lambda r: r/r.sum(), axis=1)
confusion_matrix.head()

In [ ]:
cm2 = pd.crosstab(m['EXT_CATEGORIZATION'], m['Pathogenicity'], rownames=['Clinical Review'], colnames=['Azurify']).apply(lambda r: r/r.sum(), axis=1)
cm2.head()

In [30]:
# THIS IS THE CANCER VAR SET WITH NO Benign USING THIS FOR PUBLICATION 

# NA VALUE FOR CANCERVAR IS 

#lets manually create a dataframe with the results from eaach % concordant 
#the order is Azurify first, then cancervar. class order is -> Pathogenic (disease associated), Likely Pathogenic (Probably DA), VUS (VOUS), Benign
data = { 'Azurify' : [98.53,100.00, 0.0],
        'CancerVar': [95.56, 89.97, 10.95],
        'Category': ['Pathogenic', 'Non-Pathogenic', 'N/A']}
az = pd.DataFrame(data)
az = az.set_index('Category')
az.head()

,Azurify,CancerVar
Category,,
Pathogenic,98.53,95.56
Non-Pathogenic,100.00,89.97
N/A,0.00,10.95


In [36]:

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
color_red = '#990000'  
color_blue = '#011F5B'
index = az.index
column0 = az['Azurify']
column1 = az['CancerVar']
title0 = 'Azurify'
title1 = 'CancerVar'

In [ ]:

fig, axes = plt.subplots(figsize=(10,5), ncols=2, sharey=True)
fig.tight_layout()

axes[0].barh(index, column0, align='center', color=color_blue, zorder=10)
axes[0].set_title(title0, fontsize=18, pad=15, color=color_blue)
axes[1].barh(index, column1, align='center', color=color_red, zorder=10)
axes[1].set_title(title1, fontsize=18, pad=15, color=color_red)
axes[0].invert_xaxis()
axes[0].set(yticks=az.index, yticklabels=az.index)
axes[0].yaxis.tick_left()
axes[0].tick_params(axis='y', colors='white') # tick color

axes[1].set_xticks([0, 20, 40, 60, 80, 100])

for label in (axes[0].get_xticklabels() + axes[0].get_yticklabels()):
    label.set(fontsize=16, color='black')
for label in (axes[1].get_xticklabels() + axes[1].get_yticklabels()):
    label.set(fontsize=16, color='black')
#plt.style.use('seaborn-white')
plt.gca().invert_yaxis()
plt.subplots_adjust(wspace=0, top=0.85, bottom=0.1, left=0.18, right=0.95)
plt.savefig('AZvsCV.all_concordant.pdf', dpi=600)

In [ ]:
#now lets read in all of the uab and uc data
uab_cv_aml = pd.read_csv('UAB_cancervar_aml.txt', sep='\t', header=None, names=['CHROM', 'POS', 'REF', 'ALT', 'CancerVar'])
uab_cv_lung = pd.read_csv('UAB_cancervar_lung.txt', sep='\t', header=None, names=['CHROM', 'POS', 'REF', 'ALT', 'CancerVar'])
UC_cv_all = pd.read_csv('UC_cancervar_all.txt', sep='\t', header=None, names=['CHROM', 'POS', 'REF', 'ALT', 'CancerVar'])
ext_cv_all = pd.concat([uab_cv_aml, uab_cv_lung, UC_cv_all])
ext_cv_all.head()

In [43]:
#first lets harmonize data from the cancervar calls
cv = ext_cv_all.rename({0: 'CHROM', 1: 'POS', 2: 'REF', 3: 'ALT', 4: 'CancerVar'}, axis=1)
cv['CHROM'] = 'chr' + cv['CHROM']
#'Tier I/Pathogenic: Variants of Strong Clinical Significance','Tier II/Likely Pathogenic: Variants of Potential Clinical Significance','Tier III: Variants of Unknown Clinical Significance' and 'Tier IV: Benign or Likely Benign Variants', together with detailed evidence.
#lets convert the cancervar to cpd cats
cv['CancerVar'] = cv['CancerVar'].str.split('#').str[1]
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_I_strong', 'Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_III_Uncertain', 'Non-Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_II_potential', 'Pathogenic')
cv['CancerVar'] = cv['CancerVar'].astype(str).str.replace('Tier_IV_benign', 'Non-Pathogenic')

In [105]:
#read in all azurify data
uc_az_all = pd.read_csv('UC_azurify.full.tsv', sep='\t')
uc_az_all = uc_az_all.rename(columns={'UC_CATEGORIZATION': 'EXT_CATEGORIZATION'})
uc_az_all.drop(['DIAGNOSIS'], axis=1, inplace=True)
uab_aml =  pd.read_csv('UAB.aml.azurify.tsv', sep='\t')
uab_aml = uc_az_all.rename(columns={'UAB_CATEGORIZATION': 'EXT_CATEGORIZATION'})
#uab_aml.drop(['Diagnosis'], axis=1, inplace=True)
uab_lung = pd.read_csv('UAB.lung.azurify.tsv', sep='\t')
uab_lung = uab_lung.rename(columns={'UAB_CATEGORIZATION': 'EXT_CATEGORIZATION'})
uab_lung.drop(['Diagnosis'], axis=1, inplace=True)
ext_az_all = pd.concat([uc_az_all, uab_aml, uab_lung])

In [111]:
az = az.replace(to_replace='Likely Pathogenic', value='Pathogenic', regex=True)
az = az.replace(to_replace='VUS', value='Non-Pathogenic', regex=True)
az = az.replace(to_replace='Likely Benign', value='Non-Pathogenic', regex=True)
az = az.replace(to_replace='Benign', value='Non-Pathogenic', regex=True)
az = az.replace(to_replace='Likely Non-Pathogenic', value='Non-Pathogenic', regex=True)


In [ ]:
m = az.merge(cv, how='left', on=['CHROM','POS', 'REF', 'ALT'])
m.drop_duplicates()

In [ ]:
confusion_matrix = pd.crosstab(m['EXT_CATEGORIZATION'], m['CancerVar'], rownames=['EXT_CATEGORIZATION'], colnames=['CancerVar']).apply(lambda r: r/r.sum(), axis=1)
confusion_matrix.head()

In [ ]:
cm2 = pd.crosstab(m['EXT_CATEGORIZATION'], m['Pathogenicity'], rownames=['Clinical Review'], colnames=['Azurify']).apply(lambda r: r/r.sum(), axis=1)
cm2.head()

In [119]:
# THIS IS THE CANCER VAR SET WITH NO Benign USING THIS FOR PUBLICATION 

# NA VALUE FOR CANCERVAR IS 

#lets manually create a dataframe with the results from eaach % concordant 
#the order is Azurify first, then cancervar. class order is -> Pathogenic (disease associated), Likely Pathogenic (Probably DA), VUS (VOUS), Benign
data = { 'Azurify' : [92.92, 99.55, 0.0],
        'CancerVar': [52.4, 89.62, 30.02],
        'Category': ['Pathogenic', 'Non-Pathogenic', 'N/A']}
az2 = pd.DataFrame(data)
az2 = az2.set_index('Category')
az2.head()

,Azurify,CancerVar
Category,,
Pathogenic,92.92,52.40
Non-Pathogenic,99.55,89.62
N/A,0.00,30.02


In [120]:

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
color_red = '#990000'  
color_blue = '#011F5B'
index = az2.index
column0 = az2['Azurify']
column1 = az2['CancerVar']
title0 = 'Azurify'
title1 = 'CancerVar'

In [ ]:

fig, axes = plt.subplots(figsize=(10,5), ncols=2, sharey=True)
fig.tight_layout()

axes[0].barh(index, column0, align='center', color=color_blue, zorder=10)
axes[0].set_title(title0, fontsize=18, pad=15, color=color_blue)
axes[1].barh(index, column1, align='center', color=color_red, zorder=10)
axes[1].set_title(title1, fontsize=18, pad=15, color=color_red)
axes[0].invert_xaxis()
axes[0].set(yticks=az2.index, yticklabels=az2.index)
axes[0].yaxis.tick_left()
axes[0].tick_params(axis='y', colors='white') # tick color

axes[1].set_xticks([0, 20, 40, 60, 80, 100])

for label in (axes[0].get_xticklabels() + axes[0].get_yticklabels()):
    label.set(fontsize=16, color='black')
for label in (axes[1].get_xticklabels() + axes[1].get_yticklabels()):
    label.set(fontsize=16, color='black')
#plt.style.use('seaborn-white')
plt.gca().invert_yaxis()
plt.subplots_adjust(wspace=0, top=0.85, bottom=0.1, left=0.18, right=0.95)
plt.savefig('AZvsCV.all.pdf', dpi=600)